In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
property_list = pd.read_csv("all_property_details.csv")
property_list = property_list.replace(to_replace='None', value=np.nan)
property_list = property_list.replace(to_replace='✔', value=1)
# Calculate the percentage of NaN values per column
nan_percentages = (property_list.isna().sum() / len(property_list)) * 100
# Filter columns where percentage of NaN is 60% or more
columns_to_remove = nan_percentages[nan_percentages >= 45].index
# Remove the filtered columns
property_list = property_list.drop(columns=columns_to_remove)

In [3]:
sold_homes = property_list[property_list['Price'] == 'Sold']

In [4]:
# Create a new column to identify whether the home is active or sold
property_list['Status'] = property_list['Price'].apply(lambda x: 'Sold' if x == 'Sold' else 'active')

# Group by Area and Status, and count the occurrences
grouped_df = property_list.groupby(['Area', 'Status']).size().unstack(fill_value=0)

In [5]:
grouped_df['Inventory'] = grouped_df['active']/grouped_df['Sold']

In [7]:
# Sort the DataFrame by 'Inventory' column in descending order
sorted_df = grouped_df.sort_values(by='Inventory', ascending=False)

# Create a Plotly horizontal bar chart
fig = go.Figure()

# Add a bar trace
fig.add_trace(go.Bar(
    x=sorted_df['Inventory'],  # Inventory values
    y=sorted_df.index,  # Areas
    orientation='h',  # Horizontal bar chart
    marker_color='blue',
    text=sorted_df['Inventory'].round(2),  # Round to 2 decimal places for visual clarity
    textposition='auto',
    texttemplate='%{text}',  # Show numbers on bars
    name='Inventory'
))

# Update layout
fig.update_layout(
    title='Inventory by Area (Ranked)',
    xaxis=dict(title='Inventory'),
    yaxis=dict(title='Area', autorange="reversed"),  # Reverse the y-axis for ranking
    template='plotly_white',
    bargap=0.15  # Set the gap between bars
)

# Show the plot
fig.show()